# `01` Import Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


In [2]:

ratings = pd.read_csv(r"C:\Users\mohamed elsayed\Desktop\ml-latest-small\Movie Recommendation System app\dataset\ratings.csv")
movies = pd.read_csv(r"C:\Users\mohamed elsayed\Desktop\ml-latest-small\Movie Recommendation System app\dataset\movies.csv")


In [3]:
movie_data = pd.merge(ratings, movies, on='movieId')


In [4]:

movies["year"]=movies["title"].str.extract(r'\((\d{4})\)')
movies["year"].fillna(1995, inplace=True)
movies["year_cat"]=pd.cut(movies["year"].astype(int), bins=[1900, 1970, 1990, 2000, 2010, 2020], labels=["1900-1970", "1970-1990", "1990-2000", "2000-2010", "2010-2020"])
year_cat=pd.get_dummies(movies["year_cat"],dtype=int)   
year_cat["movieId"]=movies["movieId"] 

C:\Users\mohamed elsayed\AppData\Local\Temp\ipykernel_6884\3742186625.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies["year"].fillna(1995, inplace=True)


In [5]:
  
movie_genres = movies.set_index('movieId')['genres'].str.get_dummies(sep='|')


In [6]:
utility_matrix= movie_data.pivot_table(index='movieId', columns='userId', values='rating')

In [7]:
utility_matrix.fillna(0, inplace=True)


In [8]:

final=utility_matrix.merge(movie_genres, on='movieId', how='inner')
final=final.merge(year_cat, on='movieId', how='inner')
final.set_index('movieId', inplace=True)


In [9]:
similarity=cosine_similarity(final)
similarity=pd.DataFrame(similarity, index=final.index, columns=final.index,dtype=float)


In [10]:
def getsim(movieid,topn=10):
    index1=similarity[movieid].sort_values(ascending=False).index[1:topn+1]
    val=val=similarity[movieid].sort_values(ascending=False).values[1:topn+1]
    df=pd.DataFrame(val,index=index1).rename(columns={0:"similarity"}).set_index(index1)
    return df.merge(movies.set_index("movieId"), on='movieId')

In [12]:
df=getsim(1,10)
df.merge(movies.set_index("movieId"), on='movieId')

,similarity,title,genres,year,year_cat
movieId,,,,,
3114,0.573591,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,1999,1990-2000
480,0.565318,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,1993,1990-2000
780,0.563897,Independence Day (a.k.a. ID4) (1996),Action|Adventure|Sci-Fi|Thriller,1996,1990-2000
260,0.556915,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,1977,1970-1990
356,0.546832,Forrest Gump (1994),Comedy|Drama|Romance|War,1994,1990-2000
364,0.541285,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,1994,1990-2000
1210,0.540599,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi,1983,1970-1990
648,0.538554,Mission: Impossible (1996),Action|Adventure|Mystery|Thriller,1996,1990-2000
1265,0.534304,Groundhog Day (1993),Comedy|Fantasy|Romance,1993,1990-2000
